<a href="https://colab.research.google.com/github/hideyuki-ogawa/autogen-hands-on/blob/main/autogen_pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## マルチエージェントを動かすライブラリAutoGenを触ってみるハンズオン（2回目）

- Microsoftが作っている
- https://microsoft.github.io/autogen/
- チュートリアルをちょっとだけ中身を変えて行う
- ほかに有名なマルチエージェント系ライブラリ
    - LangGraph: https://langchain-ai.github.io/langgraph/

## 目次

- 前回の復習（15分くらい）
    - 基礎クラス
    - コードをllmに書かせて実行
- 新しい部分(45分くらい)
    - Tool Use
    - Conversation Patterns


## まずは前回の復習

- pip install
- 基礎クラス
- llmにコードを書かせて実行する

今回もgpt-4o-miniを使って進めていきます。     
別のllmも使えるのですが、また今度。     
(解説を作る時間がなかった・・・)

In [1]:
!pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


In [2]:
from autogen import ConversableAgent
from google.colab import userdata #コラボでAPI_KEYを扱う方法

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
# エージェントの設定を作る

model_name = 'gpt-4o-mini'
api_key = userdata.get('gpt_api_key') # ここは自分のキー名を入れてください。
llm_config = {
    'config_list': [{
        'model': model_name,
        'api_key': api_key
    }]
}

In [ ]:
# 一度だけAgentと話す

agent = ConversableAgent(
    name='greet_agent',
    llm_config=llm_config,
    human_input_mode='NEVER',
    system_message='You are a greeting agent.',
    function_map=None
)
rep = agent.generate_reply(messages=[{'role': 'user', 'content': 'おはようございます。'}])
print(rep)

[autogen.oai.client: 09-27 21:37:13] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


おはようございます！今日はどんな一日になりそうですか？


In [ ]:
# エージェント同士対話させる
# 今回はmax_turns=3を渡し、対話回数の上限を３回までに制限

llm_config1 = {
    'config_list': [{
        'model': model_name,
        'api_key': api_key,
        'temperature': 0
    }]
}
llm_config2 = {
    'config_list': [{
        'model': model_name,
        'api_key': api_key,
        'temperature': 1
    }]
}

a1 = ConversableAgent(
    name='a1',
    llm_config=llm_config1,
    code_execution_config=False,
    human_input_mode='NEVER',
    system_message='あなたはお笑いコンビの突っ込み担当です。相方が面白くなるように上手く会話してください。',
)

a2 = ConversableAgent(
    name='a2',
    llm_config=llm_config2,
    code_execution_config=False,
    human_input_mode='NEVER',
    system_message='あなたはお笑いコンビのボケ担当です。提供された話題について面白いことを言って盛り上げてください。'
)

result = a1.initiate_chat(
    a2,
    message='最近選挙が多いですねぇ',
    max_turns=3
)


[autogen.oai.client: 09-27 21:42:19] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-27 21:42:20] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


a1 (to a2):

最近選挙が多いですねぇ

--------------------------------------------------------------------------------
a2 (to a1):

選挙が多いってことは、立候補者も多いってことですよね。もう、立候補者の名刺交換会やってほしいわ！「いえいえ、私も政治家ですが、名刺は渡しません。票をください！」みたいな！政策より名刺のデザインが勝負になってきたら面白いね！どこかの候補者が「私の名刺、金箔入りです！」とか言ってたら、やっぱり投票したくなっちゃうよ！

--------------------------------------------------------------------------------
a1 (to a2):

それ、確かに面白い！でも、金箔入りの名刺を持ってる候補者が「私の政策は金運アップです！」って言ったら、もうそれだけで投票しちゃうかも！でも、実際に金運アップしたら、他の候補者が「私の名刺はプラチナ入りです！」って言い出すかもしれないね。名刺バトルが始まったら、選挙じゃなくて名刺交換会になっちゃうよ！

--------------------------------------------------------------------------------
a2 (to a1):

それ、めっちゃ面白い！名刺交換会が選挙を制覇したら、逆に「名刺大臣」とかが誕生しちゃうよ！その大臣が「国民の皆さん、名刺はこうやって振りかざすと効果倍増です！」とか教えてくれたら、みんな名刺を高く掲げて歩く、名刺信者の街になっちゃう！もう街中が名刺業者の看板だらけになって、選挙カーの代わりに名刺入れカーが走ってるって想像したら、ちょっと面白い！

--------------------------------------------------------------------------------
a1 (to a2):

それ、最高だね！名刺入れカーが「皆さん、名刺をお持ちですか？今なら特別に名刺を10枚お配りします！」って走り回ってたら、もう選挙どころじゃないよ！みんな名刺を交換するために行列作って、名刺のデザインコンテストとか始まったら、

In [ ]:
result.chat_history

[{'content': '最近選挙が多いですねぇ', 'role': 'assistant', 'name': 'a1'},
 {'content': '選挙が多いってことは、立候補者も多いってことですよね。もう、立候補者の名刺交換会やってほしいわ！「いえいえ、私も政治家ですが、名刺は渡しません。票をください！」みたいな！政策より名刺のデザインが勝負になってきたら面白いね！どこかの候補者が「私の名刺、金箔入りです！」とか言ってたら、やっぱり投票したくなっちゃうよ！',
  'role': 'user',
  'name': 'a2'},
 {'content': 'それ、確かに面白い！でも、金箔入りの名刺を持ってる候補者が「私の政策は金運アップです！」って言ったら、もうそれだけで投票しちゃうかも！でも、実際に金運アップしたら、他の候補者が「私の名刺はプラチナ入りです！」って言い出すかもしれないね。名刺バトルが始まったら、選挙じゃなくて名刺交換会になっちゃうよ！',
  'role': 'assistant',
  'name': 'a1'},
 {'content': 'それ、めっちゃ面白い！名刺交換会が選挙を制覇したら、逆に「名刺大臣」とかが誕生しちゃうよ！その大臣が「国民の皆さん、名刺はこうやって振りかざすと効果倍増です！」とか教えてくれたら、みんな名刺を高く掲げて歩く、名刺信者の街になっちゃう！もう街中が名刺業者の看板だらけになって、選挙カーの代わりに名刺入れカーが走ってるって想像したら、ちょっと面白い！',
  'role': 'user',
  'name': 'a2'},
 {'content': 'それ、最高だね！名刺入れカーが「皆さん、名刺をお持ちですか？今なら特別に名刺を10枚お配りします！」って走り回ってたら、もう選挙どころじゃないよ！みんな名刺を交換するために行列作って、名刺のデザインコンテストとか始まったら、もう政治の話なんてどこへやら！「私の名刺、実はQRコード付きなんです！」って言ったら、若者たちが一斉にスマホをかざして、名刺の中身をチェックし始めるとか、未来の選挙は名刺で決まる時代だね！',
  'role': 'assistant',
  'name': 'a1'},
 {'content': 'ほんとにそうなっちゃうね！未来の選挙

In [ ]:
result.cost

{'usage_including_cached_inference': {'total_cost': 0.0007202999999999999,
  'gpt-4o-mini-2024-07-18': {'cost': 0.0007202999999999999,
   'prompt_tokens': 1654,
   'completion_tokens': 787,
   'total_tokens': 2441}},
 'usage_excluding_cached_inference': {'total_cost': 0.0007202999999999999,
  'gpt-4o-mini-2024-07-18': {'cost': 0.0007202999999999999,
   'prompt_tokens': 1654,
   'completion_tokens': 787,
   'total_tokens': 2441}}}

## コードをllmに書かせて実行する

In [ ]:
import tempfile
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor
import datetime

In [ ]:
code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""

llm_config1 = {
    'config_list': [{
        'model': model_name,
        'api_key': api_key,
        'temperature': 0
    }]
}

executor = LocalCommandLineCodeExecutor(
    timeout=10,
    work_dir='/content'
)

code_executor_agent = ConversableAgent(
    name='code_executor',
    llm_config=False,
    code_execution_config={
        'executor': executor},
    human_input_mode='ALWAYS'
)

code_writer_agent = ConversableAgent(
    name='code_writer',
    llm_config=llm_config1,
    system_message=code_writer_system_message,
    code_execution_config=False,
    human_input_mode='NEVER'
)

today = datetime.datetime.now().strftime('%Y-%m-%d')
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=f'今日は{today}です。MSFT と META の株価の年始からの変化率を算出してください。そしてそのグラフを "stock-gains.png" という名前で保存してください。'
)



[autogen.oai.client: 09-27 21:54:04] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


code_executor (to code_writer):

今日は2024-09-27です。MSFT と META の株価の年始からの変化率を算出してください。そしてそのグラフを "stock-gains.png" という名前で保存してください。

--------------------------------------------------------------------------------
code_writer (to code_executor):

まず、MSFT（Microsoft）とMETA（Meta Platforms）の株価データを取得し、年始からの変化率を計算します。その後、変化率をグラフにプロットして "stock-gains.png" という名前で保存します。

以下の手順で進めます：

1. Yahoo Finance APIを使用してMSFTとMETAの株価データを取得します。
2. 年始の株価と現在の株価を比較して変化率を計算します。
3. 変化率をグラフにプロットし、画像ファイルとして保存します。

次のPythonコードを実行してください。

```python
# filename: stock_analysis.py
import yfinance as yf
import matplotlib.pyplot as plt

# 株式シンボル
symbols = ['MSFT', 'META']

# 年始の日付
start_date = '2024-01-01'
end_date = '2024-09-27'

# データを取得
data = {symbol: yf.download(symbol, start=start_date, end=end_date)['Close'] for symbol in symbols}

# 年始の株価と現在の株価を取得
start_prices = {symbol: data[symbol].iloc[0] for symbol in symbols}
current_prices = {symbol: data[symbol].iloc[-1] for symbol in symbols}

# 変化率を計算
change_rates = {symbol: (cu

# ５分、自分で触ってみましょう


# では、新しい内容をスタート

- Tool Use  <===
- Conversation Patterns

## Tool Use

- Function Calling 的な奴です
- https://microsoft.github.io/autogen/docs/tutorial/tool-use

In [4]:
from autogen import register_function

In [5]:
def plus_one(a: int) -> int:
    '''整数に1を足す関数'''
    if type(a) != int:
        raise ValueError('input must be an integer or float.')
    else:
        return a + 1

llm_config = {
    'config_list': [{
        'model': model_name,
        'api_key': api_key
    }]
}

assistant = ConversableAgent(
    'assistant',
    llm_config=llm_config,
    system_message='あなたは優秀なアシスタントです'
    '私たちの計算を手伝ってください'
    '仕事が終わったら "TERMINATE" と返してください'
)
user_proxy = ConversableAgent(
    'user_proxy',
    llm_config=False,
    human_input_mode='NEVER',
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda msg: msg['content'] is not None and "TERMINATE" in msg['content']
)

# assistant.register_for_llm(name='plus_one', description='整数に1を足す関数')(plus_one)
# user_proxy.register_for_execution(name='plus_one')(plus_one)
register_function(
    plus_one,
    caller=assistant,
    executor=user_proxy,
    name='plus_one',
    description='整数に1を足す関数'
)

chat_result = user_proxy.initiate_chat(
    assistant,
    message='39に4を足してください。'
)

[autogen.oai.client: 09-28 07:11:47] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-28 07:11:47] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


user_proxy (to assistant):

39に4を足してください。

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to user_proxy):

***** Suggested tool call (call_ZBsD7HZtdQQJsp381G94IhMM): plus_one *****
Arguments: 
{"a": 39}
*************************************************************************
***** Suggested tool call (call_05Y57gWUFBSYnnO5ryyVfzPu): plus_one *****
Arguments: 
{"a": 40}
*************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION plus_one...

>>>>>>>> EXECUTING FUNCTION plus_one...
user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool (call_ZBsD7HZtdQQJsp381G94IhMM) *****
40
**********************************************************************

--------------------------------------------------------------------------------
user_proxy (to assistant):

***

In [6]:
assistant.llm_config['tools']

[{'type': 'function',
  'function': {'description': '整数に1を足す関数',
   'name': 'plus_one',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer', 'description': 'a'}},
    'required': ['a']}}}]

In [7]:
chat_result.cost

{'usage_including_cached_inference': {'total_cost': 8.039999999999999e-05,
  'gpt-4o-mini-2024-07-18': {'cost': 8.039999999999999e-05,
   'prompt_tokens': 252,
   'completion_tokens': 71,
   'total_tokens': 323}},
 'usage_excluding_cached_inference': {'total_cost': 8.039999999999999e-05,
  'gpt-4o-mini-2024-07-18': {'cost': 8.039999999999999e-05,
   'prompt_tokens': 252,
   'completion_tokens': 71,
   'total_tokens': 323}}}

## 対話パターン

- ２つのエージェントの対話
    - これは今まで見てきたので飛ばします
- 連続対話
- グループ対話
- ネスト対話


まずは連続対話から

### 連続対話

２つのエージェント間の対話が連続する。キャリーオーバーの機能により、対話がつながれる。

- ここの画像で確認: https://microsoft.github.io/autogen/docs/tutorial/conversation-patterns#sequential-chats

Aさんが、B, C, D, Eさんと次々と対話。対話内容はキャリーオーバーされるため、その内容を反映して対話が続く（よくあるチャットアプリみたいな感じ）

In [18]:
model_name = 'gpt-4o'
llm_config = {
    'config_list': [{
        'model': model_name,
        'api_key': api_key,
        'temperature': 0
    }]
}

number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

chat_results = number_agent.initiate_chats(
    [
        {
            "recipient": adder_agent,
            "message": "14",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": multiplier_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": subtracter_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": divider_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
    ]
)

[autogen.oai.client: 09-28 07:25:53] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-28 07:25:53] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-28 07:25:54] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-28 07:25:54] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-28 07:25:54] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Adder_Agent):

14

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

15

--------------------------------------------------------------------------------
Number_Agent (to Adder_Agent):

15

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

16

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Multiplier_Agent):

These are my numbers
Context: 
16

--------------------------------------------------------------------------------
Mul

In [19]:
for num, chat_result in enumerate(chat_results):
    print(f'{num}: {chat_result.summary}')

0: 16
1: 64
2: 13
61
3: 4
16
3.25
15.25


### グループチャット

- https://microsoft.github.io/autogen/docs/tutorial/conversation-patterns#group-chat

2つのエージェント以上との対話パターン。複数のエージェントのコラボレーションが必要なタスクに役立つ。

- 登場人物
    - グループチャットマネージャー
    - Agent A - D

- グループチャットマネージャーがタスクをどれかのエージェントに振る
- エージェントから答えが返される
- グループチャットマネージャーが全員にそれを伝える

というのがループする。

グループチャットマネージャーが話者を選ぶ方法がいくつかある。

- round_robin: 指定されたエージェントの順番に従い、ラウンドロビン方式でエージェントを選択します
- random: ランダムに選択
- manual: 人間の入力を求めてエージェントを選択します。
- auto: デフォルトの戦略で、Group Chat ManagerのLLMを使用してエージェントを選択します。

In [20]:
# グループチャットマネージャーが次のエージェントを選ぶのを助けるために、エージェントの概要を付ける
# autoでやる場合必要。descriptionがない場合、グループチャットマネージャーはエージェントのシステムメッセージを
# 参考にエージェントを選ぶが、ベストチョイスではない。
# descriptionは ConversableAgentでも設定できる・

adder_agent.description = "Add 1 to each input number."
multiplier_agent.description = "Multiply each input number by 2."
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."
number_agent.description = "Return the numbers given."

In [23]:
from autogen import GroupChat, GroupChatManager

model_name = 'gpt-4o-mini'
llm_config = {
    'config_list': [{
        'model': model_name,
        'api_key': api_key,
        'temperature': 0
    }]
}


number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

groupchat = GroupChat(
    agents=[number_agent, adder_agent, multiplier_agent, subtracter_agent, divider_agent],
    messages=[],
    max_round=6
)
group_chat_manager = GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
)

chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message='my number is 3, I want turn it into 13',
    summary_method='reflection_with_llm'
)

[autogen.oai.client: 09-28 07:31:01] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-28 07:31:01] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-28 07:31:02] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-28 07:31:02] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-28 07:31:02] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


...


Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4  
5  
6  
7  
8  
9  
10  
11  
12  
13  

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 07:31:02] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

6  
8  
10  
12  
14  
16  
18  
20  
22  
24  

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 07:31:10] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

2  
3  
4  
5  
6  
7  
8  
9  
10  
11  

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 07:31:10] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Divider_Agent

Divider_Agent (to chat_manager):

1.5  
2  
3  
3.5  
4  
4.5  
5  
5.5  
6  
6.5  

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 07:31:10] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Number_Agent

Number_Agent (to chat_manager):

3  
4  
5  
6  
7  
8  
9  
10  
11  
12  
13  
6  
8  
10  
12  
14  
16  
18  
20  
22  
24  
2  
3  
4  
5  
6  
7  
8  
9  
10  
11  
1.5  
2  
3  
3.5  
4  
4.5  
5  
5.5  
6  
6.5  

--------------------------------------------------------------------------------


In [24]:
print(chat_result.summary)

The conversation involves a series of mathematical operations to transform the number 3 into 13. The Adder_Agent provides a sequence of numbers leading up to 13 by adding increments, while the Multiplier_Agent lists multiples of 2 starting from 6. The Subtracter_Agent offers a range of numbers obtained by subtracting from 3, and the Divider_Agent presents results from dividing 3 by various factors. Each agent contributes different methods to reach or approach the target number of 13.


In [25]:
# 上の状態だと、グループチャットマネージャーにしか、自己紹介が役立たない
# GroupChat の send_introductionsに Trueを渡すと、参加するエージェントに自己紹介が伝わる。
# グループチャットマネージャーが各エージェントのdescriptionを会話の最初に送る

group_chat_with_introductions = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
    send_introductions=True,
)

group_chat_manager_with_intros = GroupChatManager(
    groupchat=group_chat_with_introductions,
    llm_config=llm_config
)

# Start a sequence of two-agent chats between the number agent and
# the group chat manager.
chat_result = number_agent.initiate_chats(
    [
        {
            "recipient": group_chat_manager_with_intros,
            "message": "My number is 3, I want to turn it into 13.",
        },
        {
            "recipient": group_chat_manager_with_intros,
            "message": "Turn this number to 32.",
        },
    ]
)

[autogen.oai.client: 09-28 07:33:30] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 07:33:30] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


/usr/local/lib/python3.10/dist-packages/autogen/agentchat/chat.py:53: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(



Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

3  


--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 07:33:31] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 07:33:32] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


...


Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

3

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 07:33:36] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

4

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 07:33:37] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

1

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 07:33:38] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Divider_Agent

Divider_Agent (to chat_manager):

1

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 07:33:38] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Number_Agent

Number_Agent (to chat_manager):

4  
8  
0  
0.5  

--------------------------------------------------------------------------------


### Nested Chats

- 複数のエージェントや処理を組み合わせて、段階的に作業を進める
- それにより、複雑な作業にも対応できる
- 多分 [o1](https://openai.com/index/introducing-openai-o1-preview/) なんかもこういうことなんだろうと勝手に思っている。

https://microsoft.github.io/autogen/docs/tutorial/conversation-patterns#nested-chats

In [26]:
llm_config = {
    'config_list': [{
        'model': model_name,
        'api_key': api_key,
        'temperature': 0
    }]
}

arithmetic_agent = ConversableAgent(
    name='arithmetic_agent',
    llm_config=False,
    human_input_mode='ALWAYS',
    code_execution_config={'use_docker': False, 'work_dir': '/content'}
)

code_writer_agent = ConversableAgent(
    name='code_writer_agent',
    llm_config=llm_config,
    system_message="You are a code writer. You write Python script in Markdown code blocks.",
    human_input_mode='NEVER',
)

poetry_agent = ConversableAgent(
    name='poetry_agent',
    llm_config=llm_config,
    human_input_mode='NEVER',
    system_message='You are an AI poet.'
)

[autogen.oai.client: 09-28 08:06:21] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-28 08:06:21] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [28]:
# Nested Chatを連続チャットをつかっ作る

nested_chats = [
    {
    'recipient': group_chat_manager_with_intros,
    'summary_method': 'reflection_with_llm',
    'summary_prompt': 'Summarize the sequence of operations used to turn'
    },
    {
        'recipient': code_writer_agent,
        'message': 'Write a Python script to verify the arithmetic operations is correct.',
        'summary_method': 'reflection_with_llm',
    },
    {
        'recipient': poetry_agent,
        'message': 'write a poem about it.',
        'max_turns': 1,
        'summary_method': 'last_msg'
    }
]

In [29]:
arithmetic_agent.register_nested_chats(
    nested_chats,
    trigger=lambda sender: sender not in [group_chat_manager_with_intros, code_writer_agent, poetry_agent],
)

In [31]:
reply = arithmetic_agent.generate_reply(
    messages=[{'role': 'user', 'content': 'I have a number 3 and I want to turn it into 7.'}]
)

Replying as arithmetic_agent. Provide feedback to the sender. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

********************************************************************************
Starting a new chat....

********************************************************************************
arithmetic_agent (to chat_manager):

I have a number 3 and I want to turn it into 7.

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 08:13:43] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 08:13:44] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

5

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 08:13:45] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

6

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 08:13:45] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

7

--------------------------------------------------------------------------------
[autogen.oai.client: 09-28 08:13:46] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Number_Agent

Number_Agent (to chat_manager):

8

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
arithmetic_agent (to code_writer_agent):

Write a Python script to verify the arithmetic operations is correct.
Context: 
The conversation involves a series of additions starting from the number 3, with the goal of reaching 7. The correct addition to reach 7 from 3 is 4.

--------------------------------------------------------------------------------
code_writer_agent (to arithmetic_agent):

Here is a Python script that verifies if the arithmetic operation of adding a number to 3 results in 7:

```python
def verify_addition(start, addend, expected_result):
    result = start + addend
    return result == expected_result

# Starting number
start_number = 3
#